In [32]:
import re
from navec import Navec
from scipy.spatial import distance
from sklearn.cluster import DBSCAN

ImportError: cannot import name 'getargspec_no_self'

In [2]:
def create_array_from_file(file):
    array = []
    for l in file:
        k = re.split('\s+', l)
        array.append(k[0])
    return array

In [3]:
#получение всех возможных разбиений аббревиатуры на блоки
def segmentation(abbreviation, n=10, max_len=6):
    n = min(n, len(abbreviation))
    parts_set = []
    if n==1 or len(abbreviation)==1:
        return [[abbreviation]]
    for i in range(1, min(max_len+1, len(abbreviation))):
        pref = [abbreviation[0:i]]
        parts = segmentation(abbreviation[i:len(abbreviation)], n-1)
        if pref!=['']:
            for j in range(len(parts)):
                parts[j] = pref+parts[j]
        parts_set += parts
    if len(abbreviation)<=max_len:
        parts_set += [[abbreviation]]
    return parts_set 

In [4]:
def rationality_filtration(parts_set):
    alpha = 0.8
    beta = 1 - alpha
    threshold = 0.8
    effective_parts_set = []
    for partition in parts_set:
        numerator = 0
        denumerator = 0
        for part in partition:
            completeness = int(len(part)>3)
            numerator += len(part)*completeness
            denumerator += len(part)
        R = alpha/len(partition) + beta*(numerator/denumerator)
        #print(partition)
        print(R)
        if R>=threshold:
            effective_parts_set.append(partition)
    return effective_parts_set

In [5]:
#получение всех значимых разбиений аббревиатуры
def get_reasonable_segments(abbreviation):
    parts_set = segmentation(abbreviation)
    rus_dict = open("../dictionaries/dictionary.txt", 'r', encoding='utf-8')
    rus_dict_array = create_array_from_file(rus_dict)
    list_of_extended_words_lists = []
    for partition in parts_set:
        #print(partition)
        partition_is_reasonable = True
        extended_words_list = []
        for part in partition:
            l = 0
            r = len(rus_dict_array)
            #ищём в словаре место для данного блока
            while(l < r-1):
                m = (l+r)//2
                if rus_dict_array[m] <= part:
                    l = m
                else:
                    r = m
            #print('hi')
            if rus_dict_array[l]==part:
                start = l
            else:
                start = l+1
            #находим все слова, являющиеся расширением данного блока
            words = []
            while start<len(rus_dict_array):
                word = rus_dict_array[start]
                if word[0:len(part)]==part:
                    words.append(word)
                    start += 1
                else:
                    break
            if len(words)==0:
                partition_is_reasonable = False
                break
            else:
                extended_words_list.append(words)
        if partition_is_reasonable:
            #print(extended_words_list)
            list_of_extended_words_lists.append(extended_words_list)
    return list_of_extended_words_lists         

In [6]:
def add_vectors(v, w):
    res = [(vi + wi)/2 for vi, wi in zip(v, w)]
    return res

In [15]:
def recursive_checking(part, navec, threashold, idx=0, vec=[]):
    eps = 1e-9
    if idx==len(part):
        return [[]]
    suitable = []
    for i in range(len(part[idx])):
        try:
            word_vec = navec[part[idx][i]]
        except KeyError:
            continue
        if vec!=[]:
            dist = distance.cosine(vec, word_vec)
            if dist+eps>threashold:
                continue
            new_vec = add_vectors(vec, word_vec)
        else:
            new_vec = word_vec
        word = [part[idx][i]]    
        phrases = recursive_checking(part, navec, threashold, idx+1, new_vec)
        for j in range(len(phrases)):
            phrases[j] = word+phrases[j]
        suitable += phrases
    return suitable

In [16]:
def segments_union(partition, threashold):
    path = '../dataset/fit_language_models/navec_hudlit_v1_12B_500K_300d_100q.tar'
    navec = Navec.load(path)
    phrases = []
    for part in partition:
        phrases += recursive_checking(part, navec, threashold)
    return phrases

In [ ]:
def clasterization(phrases):
    

In [10]:
# parts_set = segmentation('даздрасперма')
# rationality_filtration(parts_set)

In [20]:
k = get_reasonable_segments('тд')

In [22]:
segments_union(k, 0.7)

C:\Users\anna_\anaconda3\envs\m\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


[['табак', 'древесный'],
 ['табак', 'дрова'],
 ['табак', 'дрянь'],
 ['табак', 'дурь'],
 ['табак', 'душистый'],
 ['табак', 'дым'],
 ['табак', 'дымок'],
 ['табакерка', 'дароносица'],
 ['табакерка', 'дарохранительница'],
 ['табакерка', 'дарственная'],
 ['табакерка', 'дачка'],
 ['табакерка', 'диадема'],
 ['табакерка', 'дощечка'],
 ['табакерка', 'драгоценность'],
 ['табакерка', 'драхма'],
 ['табакерка', 'дублон'],
 ['табакерка', 'дудка'],
 ['табакерка', 'дудочка'],
 ['табакерка', 'дукат'],
 ['табачник', 'далматинец'],
 ['табачник', 'дальнозоркий'],
 ['табачник', 'данник'],
 ['табачник', 'двуединый'],
 ['табачник', 'двужильный'],
 ['табачник', 'двунадесятый'],
 ['табачник', 'дебелый'],
 ['табачник', 'дебютант'],
 ['табачник', 'девяностолетний'],
 ['табачник', 'девятиклассник'],
 ['табачник', 'дегтярник'],
 ['табачник', 'дедуся'],
 ['табачник', 'дедушкин'],
 ['табачник', 'дергун'],
 ['табачник', 'десятиклассник'],
 ['табачник', 'детдомовец'],
 ['табачник', 'дефлектор'],
 ['табачник', 'дехкани

ERROR: Could not find a version that satisfies the requirement xml
ERROR: No matching distribution found for xml


In [23]:
k = get_reasonable_segments('йюй')

In [24]:
segments_union(k, 0.7)

C:\Users\anna_\anaconda3\envs\m\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


[['йодистый', 'юнкерский', 'йод'],
 ['йодистый', 'юнкерский', 'йодистый'],
 ['йоркшир', 'юго-восточный', 'йоркшир'],
 ['йоркшир', 'юго-восточный', 'йоркширский'],
 ['йоркшир', 'юго-западный', 'йоркшир'],
 ['йоркшир', 'юго-западный', 'йоркширский'],
 ['йоркшир', 'южный', 'йоркшир'],
 ['йоркшир', 'южный', 'йоркширский'],
 ['йоркширский', 'южноамериканский', 'йодистый'],
 ['йоркширский', 'южноамериканский', 'йоркшир'],
 ['йоркширский', 'южноамериканский', 'йоркширский'],
 ['йоркширский', 'южноафриканский', 'йоркшир'],
 ['йоркширский', 'южноафриканский', 'йоркширский'],
 ['йоркширский', 'южнорусский', 'йоркширский'],
 ['йоркширский', 'юморок', 'йоркширский']]

In [25]:
a = [(5, ['а', 'б']), (6, ['а', 'б'])]

In [26]:
a.sort(key=lambda n: n[0])

In [27]:
a

[(5, ['а', 'б']), (6, ['а', 'б'])]

In [31]:
!pip install sklearn --upgrade

In [35]:
import numpy
import scipy
import sklearn

ImportError: cannot import name 'getargspec_no_self'